# Preprocessing for SPACE-MS

This notebook describes the steps that were taken to execute the SPACE-MS algorithm.

In [2]:
# Necessary imports
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import os
from scipy.ndimage import label


In [ ]:
def load_img_and_lesion(patient, visit):

    """
    Load the T1-weighted volume and corresponding T2 lesion mask of a patient at a particular time point.

    Args:
        patient (str): Patient ID formatted as a three-digit string (e.g., '001', '050', '101').
        visit (str): MRI scan time point (e.g., '1', '2', '3', or '4').

    Returns:
        nib.Nifti1Image: T1-weighted volume
        nib.Nifti1Image: T2 lesion mask    
    """

    # Paths for the T1 volume and T2 lesion mask
    path_img_t1 = f"/input_inference/01-{patient}_visit_{visit}/MS_01-{patient}_0001.nii.gz"
    path_lesion_mask = f"/output_inference/01-{patient}_visit_{visit}/MS_01-{patient}.nii.gz"

    # Load volume and lesion mask
    img_t1 = nib.load(path_img_t1)
    lesion_mask = nib.load(path_lesion_mask)

    return img_t1, lesion_mask

## Remove small lesions

Small lesions raises errors during imputation of the covariance matrix when running the SPACE-MS code run_sspace_ms.py. Therefore all lesions that were smaller than 10 voxels were removed from the lesion data.

In [1]:
def remove_lesions(patient, visit):

    """
    Generates altered T2 lesion masks where the smallest lesions (under 10 voxels) are removed

    Args:
        patient (str): Patient ID formatted as a three-digit string (e.g., '001', '050', '101').
        visit (str): MRI scan time point (e.g., '1', '2', '3', or '4').

    Returns:
        None: Saves lesion mask as NIfTI file   
    """

    # Path where the new lesion masks are saved
    output_path = f"/masks_small_lesions_removed/{patient}_visit_{visit}.nii.gz"

    # Load the lesion mask
    _, lesion_mask = load_img_and_lesion(patient, visit)

    # Label all lesions and compute number of lesions
    labeled_array, num_labels = label(lesion_mask.get_fdata())

    # Create an array of only zeros the same shape as the lesion mask
    filtered_img = np.zeros_like(labeled_array)                    

    # Create binary lesion masks containing the lesions that are larger than 10 voxels
    for i in range(1, num_labels +1):
        if (np.sum(labeled_array == i)) >= 10:
            filtered_img[labeled_array == i] = 1                  

    # Save the lesion mask
    filtered_nifti = nib.Nifti1Image(filtered_img, lesion_mask.affine)
    nib.save(filtered_nifti, output_path)


In [7]:
# Path to T2 lesions
inference_output_path = f"/output_inference/"

# Loop through patients and remove the small lesions
for patient_visit in os.listdir(inference_output_path):
    ID = patient_visit[3:6]
    visit = patient_visit[-1:]
    remove_lesions(ID, visit)